In [ ]:
import numpy as np
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

'''
uniformly sample from (dim-1) dimension unit sphere in (dim) dimension space
args:
    number: number to sample
    dim:    space dimensiion
returns:
    a (num,3) vector of the sampling results, each row represents the coordinates of a sampling point.
'''
def random_dir(number,dim = 3):
    arr = np.random.randn(number,dim)
    radius = np.sqrt(np.sum(arr * arr,axis = 1))
    return (arr.T / radius).T

'''
generate some step lengths in terms of Gaussian distribution
args:
    number:   number of step
    average:  the expectation of lengths
    variance: the Variance of lengths
returns:
    a (number,) vector representing step_lengths
'''
def random_step_length(number,average=1,variance=0.3):
    length = average+np.random.randn(number)*variance
    return length

'''
generate some radius in terms of Gaussian distribution
args:
    number:   number of radius
    average:  the expectation of radius
    variance: the Variance of radius
returns:
    a (number,) vector representing radius
'''
def random_radius(number,average=0.4,variance=0.1):
    radius = abs(average+np.random.randn(number)*variance)
    return radius

'''
generate a random train by specific dirs  generator and steplength generator
args:
    number:      the number of nodes in the chain
    dirs_gen:    the function to randomly generate the directions from a node to the next node
    lengths_gen: the function to randomly generate the lengths between adjacent nodes
    radius_gen:  the function to randomly generate radius of the nodes in chain
returns:
    the random chain, its a list of namedtuple, each item of the list is a tuple, including coordinate
        and radius
'''
Chain = namedtuple('Chain',['ps','rs'])
def random_chain(number,dirs_gen = random_dir,lengths_gen = random_step_length,radius_gen = random_radius):
    dirs = dirs_gen(number)
    step = lengths_gen(number)
    chain = Chain(np.cumsum((step*dirs.T).T,axis=0),radius_gen(number))
    return chain

def random_angle(number):
    return np.random.random(number)*np.pi

Projection = namedtuple('Projection',['ps','rs'])
def project(chain,direction,theta=0):
    direction = direction/np.sqrt(direction.dot(direction))
    tx = np.zeros_like(direction)
    tx[np.argmin(abs(direction))] = 1
    tx -= direction[np.argmin(abs(direction))]*direction
    tx = tx/np.sqrt(tx.dot(tx))
    ty = np.cross(direction,tx)
    t = np.vstack((tx,ty))
    coord = chain.ps.dot(t.T)
    coord = np.vstack((coord[:,0]*np.cos(theta)-coord[:,1]*np.sin(theta),(coord[:,0]*np.sin(theta)+coord[:,1]*np.cos(theta)))).T
    return Projection(coord,chain.rs)

def draw(projection,size=5,sample_num=51):
    x = np.linspace(-size,size,sample_num)
    y = np.linspace(-size,size,sample_num)
    xv,yv = np.meshgrid(x,y)
    vmap = np.zeros_like(xv)
    num = projection.rs.shape[0]
    for iter in range(num):
        vmap+=(1./2./np.pi/np.power(projection.rs[iter],2))*np.exp((-np.power(xv-projection.ps[iter,0],2)-np.power(yv-projection.ps[iter,1],2))/2./np.power(projection.rs[iter],2))
    vmap*=np.power(2*size/(sample_num-1),2)
    return vmap


In [ ]:
NUMBER = 13
chain = random_chain(NUMBER)
proj = project(chain,np.array([1,0,0]),0)
vmap = draw(proj,size=3,sample_num=28)
print(np.max(vmap))
uplim=0.5
vmap[vmap>uplim]=uplim
plt.imshow(vmap,cmap='gray')

In [ ]:
import cv2
rep = 20
for k in range(19):
    for t in range(rep):
        num = (k*rep+t)
        proj = project(chain,np.array([(k+1)/10-1,1,1]),0)
        vmap = draw(proj,size=3,sample_num=28)*1200
        print(np.max(vmap))
        cv2.imwrite('pic/'+str(num)+'.bmp',vmap)